In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os
import json

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

molecule_concept_selector_config = {
    "dropdown": {"width": "315px"},
    "details": {"width": "415px", "height": "250px"},
    "image": {"width": "220px", "height": "250px"}
}
molecule_concept_selector = widgets.ObjectSelectionWidget("Molecule Concept", molecule_concept_selector_config)
molecule_concept_selector.load_dropdown_box("MOLECULE_CONCEPT", "molecule concept")

chemist_selector = widgets.ObjectSelectionWidget("Chemist")
chemist_selector.load_dropdown_box("CHEMIST", "chemist")

storage_selector = widgets.ObjectSelectionWidget("Storage")
storage_selector.load_dropdown_box("STORAGE", "storage")

supplier_selector = widgets.ObjectSelectionWidget("Instrument")
supplier_selector.load_dropdown_box("SUPPLIER", "supplier")

molecule_widgets = widgets.ObjectPropertiesWidgets("Molecule")
molecule_widgets.get_properties_widgets()

molecule_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def create_molecule_openbis(b):
    if molecule_concept_selector.dropdown.value == -1:
        display(utils.Javascript(data = "alert('Select a molecule concept.')"))
        return
    
    # Check if the molecule is already in openBIS
    molecules_empa_id_openbis = [f"{molecule.props['empa_number']}-{molecule.props['batch']}" for molecule in utils.get_openbis_objects(OPENBIS_SESSION, type ="MOLECULE")]
    
    object_properties = {}
    for prop in CONFIG["objects"]["Molecule"]["properties"]:
        property_type = CONFIG["properties"][prop]["property_type"]
        
        if prop in molecule_widgets.properties_widgets:
            property_widget = molecule_widgets.properties_widgets[prop]
            object_properties[prop] = utils.get_object_property(property_type, property_widget)
    
    molecule_empa_id = f"{object_properties['empa_number']}-{object_properties['batch']}"
    if molecule_empa_id in molecules_empa_id_openbis:
        display(utils.Javascript(data = "alert('Molecule is already in openBIS!')"))
        return
    
    object_parents = []
    
    if molecule_concept_selector.dropdown.value != -1:
        object_parents.append(molecule_concept_selector.dropdown.value)
        
    if chemist_selector.dropdown.value != -1:
        object_parents.append(chemist_selector.dropdown.value)
        
    if supplier_selector.dropdown.value != -1:
        object_parents.append(supplier_selector.dropdown.value)
    
    if storage_selector.dropdown.value != -1:
        object_parents.append(storage_selector.dropdown.value)

    molecule_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="MOLECULE", 
        collection="/MATERIALS/MOLECULES/PRECURSOR_COLLECTION", 
        props = object_properties,
        parents = object_parents
    )
    utils.upload_datasets(OPENBIS_SESSION, molecule_object, molecule_support_files, "RAW_DATA")
    
    display(utils.Javascript(data = "alert('Upload successful!')"))
    molecule_widgets.reset_properties_widgets()


def checkbox_on_change(checkbox, textbox):
    def enable_textbox(change):
        if checkbox.value:
            textbox.disabled = False
        else:
            textbox.disabled = True
    checkbox.observe(enable_textbox, names = "value")

# Create molecule

## Select molecule

In [ ]:
molecule_concept_selector.dropdown.observe(molecule_concept_selector.load_metadata, names = "value")
display(molecule_concept_selector)

## Select chemist

In [ ]:
display(chemist_selector)

## Select storage

In [ ]:
display(storage_selector)

## Select supplier

In [ ]:
display(supplier_selector)

## Properties

In [ ]:
checkbox_on_change(molecule_widgets.properties_widgets["hazardous"], molecule_widgets.properties_widgets["hazardous_specification"])
checkbox_on_change(molecule_widgets.properties_widgets["other_storage_condition"], molecule_widgets.properties_widgets["other_storage_condition_specification"])
display(molecule_widgets)

## Support files

In [ ]:
display(molecule_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_molecule_openbis)
quit_button.on_click(close_notebook)